# Exercise B-Splines

This week's exercise is about a variation of linear regression and model selection. You may define auxiliary functions if you think it improves the clarity of your code.

In [1]:
# execute this cell to import all the numpy and matplotlib functions you need.
import numpy as np
import matplotlib.pyplot as plt

1. B-splines
---
Last week you implemented a linear regression using monomial basis functions. Depending on the underlying function, polynomial basis functions are not always the best choice. In the lecture B-splines were introduced, which are local and picewise polynomial functions. Piecewise polynomial simple means that it is a combination of polynomials. The following figure shows the B-spline $B_{0,3}$ with so called knots of (-3,-2,-1,0).

![b-spline](b-spline.png)

Knots or break-points are the values on the X-axis where one polynomial ends and another starts. In the figure you can see that a new polynomial starts for example at knot -3 and goes until knot -2. B-splines are also local basis function, which means that they are only non-zero on a finite intervall. The B-spline in the figure is for values smaller than -3 and bigger than 0 equal to 0. This property makes them especially usefull for certain tasks of linear regression, since you can place them exacly where you want them to have an influence. B-splines are defined recursively:

$$ B_{k,1}(x) = \left\{ \begin{matrix} 1 & \mathrm{if} \quad t_k \leq x < t_{k+1} \\ 0 & \mathrm{else} \end{matrix} \right.$$

$$B_{k,d}(x) = \frac{x - t_k}{t_{k+d-1} - t_k} B_{k,d-1}(x) + \frac{t_{k+d} - x}{t_{k+d} - t_{k+1}} B_{k+1,d-1}(x)$$

$t_k$ referes to the knot at index $k$, $d$ is the degree of the polynomial used to construct the spline. For information see script page 55.


The function `get_bspline` in the following cell, is a so called function factory (a function that returns another function). You can see that `bspline` is defined inside `get_bspline`. The last line of the outer function returns a vectorized version of the inner function, i.e. when called with an array-like value for x, the bspline function is applied elementwise to every cell. Sample usage:

```
bspline = get_bspline(knots=[0,1,2,3,4])
x = linspace(0,4,10)
y = bspline(x, k=0, d=3)
```
1. Plot all B-splines of degree three, for eleven equidistant knots between zero and ten (inclusive).
2. Given a total number of $m$ knots and an arbitrary positive integer for $d$, what is the maximal value of $k$ ? *(Note that the first knot index is 0)*
3. Try out different degrees or strange knot placements.

In [2]:
def get_bspline(knots):
    """ Takes an array-like parameter and returns a function implementing a B-spline. """
    
    def bspline(x, k, d): 
        if d==1: # recursion stop
            if knots[k] <= x < knots[k+1]:
                y = 1
            else:
                y = 0
        else:
            factor1 = (x-knots[k])/(knots[k+d-1]-knots[k])
            factor2 = (knots[k+d]-x)/(knots[k+d]-knots[k+1])
            b1 = bspline(x, k, d-1)   # recursion!
            b2 = bspline(x, k+1, d-1) # recursion!
            y = factor1*b1 + factor2*b2
        return y

    return np.vectorize(bspline, excluded=['k','d'])

In [3]:
# Your Code here.

2. Cubic B-spline linear regression
---

Implement a function that performs a linear regression using $(B_{0,d}, \dots, B_{m-d-1,d})$ as basis functions. Use `pinv` for calculating the pseudoinverse.



In [4]:
def bspline_regression(X, Y, knots, deg):
    """
    Performes a B-spline regression.
    
    Parameters
    ----------
    X : 1-D ndarray
        samples of the independent variable
    Y : 1-D ndarray
        samples of the dependent variable
    knots : 1-D array-like
        knots of the splines
    deg : integer
        order of splines
    
    Returns
    -------
    w : ndarray
        egression weights
    """

    #-- enter code here --#
    
    return w


3. Fit, choose, plot 
-----
Load the data from *BSplinesCrossValidationHyperparameters_data.csv*. You will find a 2x200 array, the first column of which is the indepent, the second column the dependent variable. Look at the data and your plot from above to find a good knot array for a B-spline regression (at this point it is not important to find the best model, we do that later). Fit a model with the knots of your choice to the data and plot the resulting graph together with a scatter plot of the data. Briefly describe why you chose those specific knots.

In [5]:
# Your Code

4. Cross-validation
----

You may have noticed that no underlying target function is given this time. This is the typical case in real applications. The most popular approach to verify your fit in this case is cross-validation. Implement a 10-fold cross-validation and validate the model you chose in task 3. Print the mean and the standard deviation of the RMSE of your predidictions versus the validation data. Make sure the partition of the data is randomized.

The following instructions are meant as a help. You may or may not follow them.

(assuming you stored your data in `X` and `Y`)
* create a list of length `len(X)` of indices
* shuffle that list
* split the indices into ten equally sized chunks
* For the chunk $c\in \{1,\dots,10\}$ do the following:
    * use the data correspondeing to the indices in $c$ as validation data (`X_val` and `Y_val`)
    * use the data corresponding to remaining 9 chunks as training data (´X_train´ and ´Y_train´)
    * fit your chosen spline model on the training data
    * make an estimation with your model on the validation data (store it in ´Y_est´)
    * calculate the RMSE between your estimation and the validation data
* Calculate the mean and standard deviation of the errors

*Functions: mean, std, ravel, delete, shuffle*

In [7]:
def rmse(x,y):
    return np.sqrt(np.sum((x-y)**2)/len(x))

# Your Code

def cross_validate(x, y, spline_model, model):
    
    # create a list of length `len(X)` of indices
    indices = np.arange(len(X))
    
    # shuffle that list
    np.random.shuffle(indices)
    
    # split the indices into ten equally sized chunks
    chunks = np.reshape(indices, (10, -1))
    
    # the error list
    errors = []
    
    # lenght of chunks for upper range for the iteration
    len_chunks = len(chunks)
    
    for index in range(len_chunks): # we have 10 setups
        
        # use the data correspondeing to the indices in  𝑐  as validation data (X_val and Y_val)
        X_val, Y_val = x[chunks[index]], y[chunks[index]]
        
        # use the data corresponding to remaining 9 chunks as training data (´X_train´ and ´Y_train´)                   
        remain = np.ravel(np.delete(chunks, index ,axis=0))
        x_train, y_train = x[remain], y[remain]

        # fit your chosen spline model on the training data
        fitted_model = spline_model(x_train, y_train)
        
        # make an estimation with your model on the validation data (store it in ´Y_est´)
        Y_est = model(X_val, fitted_model)
        
        # calculate the RMSE between your estimation and the validation data
        calculated_rmse = rmse(Y_est, Y_val)
        
        # adding the error of this chunk to the error list
        errors.append(calculated_rmse)
    
    # calculating and returning the mean and standard deviation of the errors
    return np.mean(errors), np.std(errors)


# get the mean and standard deviation of the errors
mu, sd = cross_validate(
    # all the parameters of this function are needed from the prevoius tasks to be passed to this function
)

# the below statements give an output when the above tasks are implemented
print("Mean: ", mu)
print("Standard diviation: ", sd)

TypeError: cross_validate() missing 4 required positional arguments: 'x', 'y', 'spline_model', and 'model'


5. Model selection
------
We can distinguish two kinds of parameters. The first are coefficients in the regression model; we call them regression parameters. But there are other parameters we can manipulate in order to optimize our model. In the spline model, we can change the number of knots, the knot placement and spline degree; we call them hyper paramters.


Write code that automatically finds a good set of hyperparamters, simply by trying different combinations of knot number, knot placement or spline degree. Use crossvalidation to determine the performance of a model. Print the regression weights, the knots and the spline degree of your best model to the console.

*Hint: You can assume that the underlying function is symmetric around 0 and the 'hills' and 'valleys' have equal distance. This might reduce your search space.*

In [ ]:
# Your Code